In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import joblib

In [3]:
# Load the dataset
df=pd.read_csv('../data/new_arabic_dialect_dataset.csv')
df.head()

,sentence,dialect,sentence_clean
0,غير احنا هاكا نبغو نزيدو عليها شفتي في تركيا ا...,DZ,غير احنا هاكا نبغو نزيدو عليها شفتي تركيا البر...
1,حبيبتي بقلك انسي سنان لان فانزو من يوم ماعملو ...,DZ,حبيبتي بقلك انسي سنان لان فانزو يوم ماعملو فيه...
2,لازم ما تسكتيش على غلطي,PL,لازم تسكتيش غلطي
3,أبشري ماطلبتي الا عزك منتخبنا هو منتخبكم الوعد...,QA,أبشري ماطلبتي الا عزك منتخبنا منتخبكم الوعد ال...
4,حتى بالكشخة يعلمهم بو عدنان اله يرحمه,BH,حتى بالكشخة يعلمهم بو عدنان يرحمه


In [4]:
# Drop rows containing NaN values
df = df.dropna()
df.head()

,sentence,dialect,sentence_clean
0,غير احنا هاكا نبغو نزيدو عليها شفتي في تركيا ا...,DZ,غير احنا هاكا نبغو نزيدو عليها شفتي تركيا البر...
1,حبيبتي بقلك انسي سنان لان فانزو من يوم ماعملو ...,DZ,حبيبتي بقلك انسي سنان لان فانزو يوم ماعملو فيه...
2,لازم ما تسكتيش على غلطي,PL,لازم تسكتيش غلطي
3,أبشري ماطلبتي الا عزك منتخبنا هو منتخبكم الوعد...,QA,أبشري ماطلبتي الا عزك منتخبنا منتخبكم الوعد ال...
4,حتى بالكشخة يعلمهم بو عدنان اله يرحمه,BH,حتى بالكشخة يعلمهم بو عدنان يرحمه


In [5]:
# Encode the target variable
label_encoder = LabelEncoder()
df['dialect'] = label_encoder.fit_transform(df['dialect'])

In [6]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['sentence_clean'], df['dialect'], test_size=0.2, random_state=42)

# Tokenize the sentences
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [8]:
# Pad the sequences to have the same length
max_len = 100
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [7]:
model = Sequential()
model.add(Embedding(10000, 128, input_length=max_len))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(20, activation='softmax'))

In [8]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=50, validation_data=(X_test, y_test))

# Save the model
joblib.dump(model, '../models/model_deep.pkl')

Epoch 1/10
5852/5852 [==============================] - 352s 60ms/step - loss: 1.1104 - accuracy: 0.6425 - val_loss: 1.8846 - val_accuracy: 0.4741
Epoch 2/10
5852/5852 [==============================] - 348s 59ms/step - loss: 1.0820 - accuracy: 0.6522 - val_loss: 1.9316 - val_accuracy: 0.4699
Epoch 3/10
5852/5852 [==============================] - 352s 60ms/step - loss: 1.0542 - accuracy: 0.6608 - val_loss: 1.9666 - val_accuracy: 0.4700
Epoch 4/10
5852/5852 [==============================] - 361s 62ms/step - loss: 1.0300 - accuracy: 0.6681 - val_loss: 2.0050 - val_accuracy: 0.4681
Epoch 5/10
5852/5852 [==============================] - 354s 61ms/step - loss: 1.0101 - accuracy: 0.6736 - val_loss: 2.0415 - val_accuracy: 0.4659
Epoch 6/10
5852/5852 [==============================] - 356s 61ms/step - loss: 0.9899 - accuracy: 0.6800 - val_loss: 2.0773 - val_accuracy: 0.4650
Epoch 7/10
5852/5852 [==============================] - 362s 62ms/step - loss: 0.9726 - accuracy: 0.6858 - val_loss: 2

['../models/model_deep.pkl']

In [9]:
# Evaluate the model on the test set
saved_model_deep = joblib.load('../models/model_deep.pkl')

loss, accuracy = saved_model_deep.evaluate(X_test, y_test, batch_size=50)
print("Test loss:", loss)
print("Test accuracy:", accuracy)

1171/1171 [==============================] - 10s 8ms/step - loss: 6.1814 - accuracy: 0.0892
Test loss: 6.181434631347656
Test accuracy: 0.08924359828233719


In [ ]:
new_data = ["أنا بحب اللحمة", "بدي اروح عالسينما"]
new_data_sequences = tokenizer.texts_to_sequences(new_data)
new_data_padded = pad_sequences(new_data_sequences, maxlen=max_len)
predictions = saved_model_deep.predict(new_data_padded)
print(predictions)